In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import StepLR

# Hyperparameters
batch_size = 64
learning_rate = 0.0005  # Keep the same learning rate
num_epochs = 100  # Set maximum epochs but add early stopping

# Data Augmentation and Preprocessing (more aggressive)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),  # Resize to the original 48x48 size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(48, padding=4),  # Random cropping with padding for variation
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # No saturation or hue needed for grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization is correct
])
# Load train and test datasets
train_dataset = datasets.ImageFolder(root='train', transform=transform)
test_dataset = datasets.ImageFolder(root='test', transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
]))



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the Custom CNN Model (no increase in complexity)
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()  # ReLU comes first now
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(256 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 7)
       
        self.dropout = nn.Dropout(p=0.3)
       
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)  # Apply ReLU first
        x = self.bn1(x)  # Apply BN after ReLU
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)  # Apply ReLU first
        x = self.bn2(x)  # Apply BN after ReLU
        x = self.pool(x)

        x = self.conv3(x)
        x = self.relu(x)  # Apply ReLU first
        x = self.bn3(x)  # Apply BN after ReLU
        x = self.pool(x)

        x = x.view(-1, 256 * 6 * 6)
       
        x = self.fc1(x)
        x = self.dropout(x)
       
        x = self.fc2(x)
        return x


# Initialize Model, Loss Function, and Optimizer
model = CustomCNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Keep label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Scheduler with more aggressive learning rate decay
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Adjusted step size

# Device Configuration (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to Calculate Accuracy
def calculate_accuracy(y_true, y_pred):
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    return accuracy_score(y_true, y_pred)

# Early Stopping Logic
def early_stopping(val_acc_list, patience=3):
    if len(val_acc_list) > patience:
        if all(val_acc_list[-i] <= val_acc_list[-i - 1] for i in range(1, patience + 1)):
            return True
    return False

# Evaluate the Model
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
   
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
           
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
           
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
   
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

# Training the Model with Early Stopping and LR Scheduling
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs, patience=5):
    model.train()
    val_acc_list = []
   
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
       
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
           
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
       
        # Evaluate test accuracy at the end of each epoch
        test_accuracy = evaluate_model(model, test_loader)
        val_acc_list.append(test_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%, Test Accuracy: {test_accuracy * 100:.2f}%')

        # Check for early stopping
        if early_stopping(val_acc_list, patience=patience):
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

        scheduler.step()

train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs)

Epoch [1/100], Loss: 2.2981, Train Accuracy: 29.27%, Test Accuracy: 41.84%
Epoch [2/100], Loss: 1.5487, Train Accuracy: 44.44%, Test Accuracy: 50.20%
Epoch [3/100], Loss: 1.4683, Train Accuracy: 49.05%, Test Accuracy: 53.46%
Epoch [4/100], Loss: 1.4247, Train Accuracy: 51.78%, Test Accuracy: 55.34%
Epoch [5/100], Loss: 1.3941, Train Accuracy: 53.52%, Test Accuracy: 56.34%
Epoch [6/100], Loss: 1.3332, Train Accuracy: 56.60%, Test Accuracy: 57.33%
Epoch [7/100], Loss: 1.3201, Train Accuracy: 57.15%, Test Accuracy: 58.28%
Epoch [8/100], Loss: 1.3111, Train Accuracy: 57.93%, Test Accuracy: 58.79%
Epoch [9/100], Loss: 1.2967, Train Accuracy: 58.50%, Test Accuracy: 59.64%
Epoch [10/100], Loss: 1.2913, Train Accuracy: 58.79%, Test Accuracy: 60.30%
Epoch [11/100], Loss: 1.2561, Train Accuracy: 60.91%, Test Accuracy: 61.08%
Epoch [12/100], Loss: 1.2494, Train Accuracy: 60.97%, Test Accuracy: 61.55%
Epoch [13/100], Loss: 1.2388, Train Accuracy: 61.94%, Test Accuracy: 61.40%
Epoch [14/100], Loss:

In [2]:
#Saving the Model
torch.save(model.state_dict(), 'cnn_fer2013_64percent.pth')